# Cyclistic case study
Author: Juan Martinez

## Summary
Cyclistic is a fictional startup which launched a successful bike-sharing program in 2016 with a fleet of 5,824 bikes and 692 stations across Chicago. The bikes can be picked up and returned to any station anytime. Cyclistic's marketing strategy has focused on appealing to a broad audience with flexible pricing options for single-ride, full-day, and annual passes.
The company's finance analysts have determined that annual members are more profitable than casual riders, and their goal is to convert casual riders into members. To do this, the marketing team wants to better understand the differences between the two groups and use data analysis to identify trends.

# 1.- Ask

### Key Stakeholders.
* Lily Moreno: Director of marketing and manager.
* Cyclistic marketing analytics team.
* Cyclistic executive team.

### Business task.

Identify the differences between “casual” and “member” users, in order to build marketing strategies to convince casual users to turn into members.

# 2.-Prepare

### Dataset used.
The data is public data provided by Motivate International Inc. Under a license agreement made between Lyft Bikes and Scooters, LLC. and the city of Chicago. The license can be read [here](https://ride.divvybikes.com/data-license-agreement).

### Information about our dataset.
Data is available from april 2021 to march 2022. Every dataset files are in CSV format, and divided between the months.

### Data organization.
For our purpose, we will use all the data available. 

# 3.- Process
### Loading the datasets

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
april_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202104-divvy-tripdata.csv')
may_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202105-divvy-tripdata.csv')
june_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202106-divvy-tripdata.csv')
july_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202107-divvy-tripdata.csv')
august_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202108-divvy-tripdata.csv')
september_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202109-divvy-tripdata.csv')
october_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202110-divvy-tripdata.csv')
november_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202111-divvy-tripdata.csv')
december_21 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202112-divvy-tripdata.csv')
january_22 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202201-divvy-tripdata.csv')
february_22 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202202-divvy-tripdata.csv')
march_22 = pd.read_csv('/kaggle/input/cyclistic-bike-share/202203-divvy-tripdata.csv')

### Combining the datasets

In [ ]:
df = pd.concat([april_21,may_21,june_21,july_21,august_21,september_21,october_21,november_21,december_21,january_22,february_22,march_22])
df

### Overall information about the dataset
Let's check some information about our dataset

In [ ]:
df.shape

In [ ]:
df.info()

### Cleaning

Checking null values and duplicates

In [ ]:
# Cheking for duplicate values
df['ride_id'].duplicated().sum()

In [ ]:
# Checking for null values
df.isnull().sum()

In [ ]:
df.isna().sum()

We see a high number of null values in some columns, but what percentage of the data is missing?

In [ ]:
# Calculating the percentage of null values
print('The percentage of missing values is:',round((df.isna().sum().max() * 100)/df.shape[0],2),'%')

13% of the data is missing, just counting the columns where there is the highest number of null values. 13% is a big deal and by that measure, our analysis could be considered bias.
However, this columns contain information about the name, and id of the station where the ride began and ended. Although of some value, it is not critical information for our analysis.
That's why I decide to keep these missing values, since we are not going to use these variables, instead of deleting the rows containing those missing values, which could endanger the analysis.

### Formatting columns

I'll check if the data types align with the desired formats for each columns

In [ ]:
df.info()

We see that the 'started_at' and 'ended_at' columns are Dtype objects, and should be dates. Let's change that

In [ ]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

### Deleting wrong entries
Now we want to check if there are not false dates, or dates that were wrongly recorded

In [ ]:
false_dates = df.loc[(df['started_at'] > df['ended_at'])]
false_dates.shape

There are 145 wrong entries where the 'started_at' column has a datetime value lower than the correspondent 'ended_at' value.
This is not possible since the start of the trip hast to be an earlier date and time than the end of the trip.

In [ ]:
#deleting the wrong entries 
df = df.drop(false_dates.index.to_list()).copy()

### Adding columns.
I will add some columns, like the length of the ride, the day of the week, the hour of the day, and the name of the month to use them later at the analysis phase

In [ ]:
df['ride_length_min'] = ((df['ended_at'] - df['started_at'])/pd.Timedelta(minutes=1)).astype('int32')

In [ ]:
# Creating the name of the day of the week
df['day'] = df['started_at'].dt.day_name()

# creating the hour of the day
df['hour'] = df['started_at'].dt.hour

# Creating the month column
df['month'] = df['started_at'].dt.month

df['day_of_week'] = df['started_at'].dt.dayofweek

With the recently column created, we sort the values in ascending order

In [ ]:
df.sort_values(by='ride_length_min',ascending=True)

We see there are entries in which the rides lasted less than 1 minute. We'll check what percentage of the data represent this instance.

In [ ]:
print('The percentage is:',df.query('ride_length_min == 0').shape[0] / df.shape[0] * 100)

With just a 1% percent, we can delete the rows which have less than 1 minute in ride length

In [ ]:
zero_length = (df['ride_length_min'] < 1)
zero_length_index = df.loc[zero_length].index.to_list()
df = df.drop(axis=0,index=zero_length_index, inplace=False)

In [ ]:
df.query('ride_length_min < 0')

# 4.- Analysis

Let's perform some exploratory and statistical analysis to our dataset, emphasizing the need to understand the bussines question.
The difference between member and casual users

### Describing the data
I'll take a quick look at the numerical data with the describe function

In [ ]:
round(df.describe(),2)

The interesting part is the ride_length_min variable. I see that the max value is way higher than the 75 percentile, meaning that this could be an outlier, and here could be more.

We need to take this into account in our analisis

### Number of values
I will count the number of values for type of users and rideable type

In [ ]:
# Count of member users and casual users
df['member_casual'].value_counts()

In [ ]:
# Count of rideable types
df['rideable_type'].value_counts()

### Grouping the data
I'll group the data by type of users and rideable type. This will be useful for further analysis

In [ ]:
# Grouping by type of user
user_grp = df.groupby('member_casual')
# Grouping by type of rideable
rideable_grp = df.groupby('rideable_type')

### Max values

In [ ]:
# Max ride length by user grp
print('max ride length by user group:')
print()
print(user_grp['ride_length_min'].max())

print()
print('max ride length by rideable type:')
print(rideable_grp['ride_length_min'].max())

We can see that the max for casual users, and specifically the one for docked bike is so big, making the math, 55944 is about 932 hours, I wonder if anyone is capable of riding a bike for that many hours.

That's clearly an outlier, but what about for the other type of bikes and users?

In [ ]:
df.groupby(['member_casual','rideable_type'])['ride_length_min'].max()

The maximum time for classic bike in both kind of users is 1599 which is about 26 hours, more than a day!. clearly an outlier if it's not someone doing a marathon.

And the maximum time for electric bike is 480, which translates to 8 hours. 


### Average and median values

In [ ]:
# Average ride length by user group
user_grp['ride_length_min'].mean()

In [ ]:
# Median ride length by user group
user_grp['ride_length_min'].median()

In [ ]:
# Average ride length by rideable type
rideable_grp['ride_length_min'].mean()

In [ ]:
# Median ride length by rideable type
rideable_grp['ride_length_min'].median()

In [ ]:
df.query('member_casual == "member"')['ride_length_min'].mode()

### Frequency of values

I'm going to analyze the number of repeating values within the column of the ride length, to see where exactly the majority of values are,and what percentage of the data the outliers are

In [ ]:
# I begin by creating a frequency table by grouping together the minutes in 40 bins
frequency_table = df.copy()
frequency_table['freq'] = pd.cut(df['ride_length_min'],40)
frequency = frequency_table.groupby('freq')['day_of_week'].count().reset_index
frequency

In [ ]:
print('the majority of unbiased values fall within the',round((5030055/df.shape[0] * 100),1),'% of the data')

the max value which is part of the 99.9% of the data is 1399.57 minutes, that's still 23 hours in length for a ride. we can still filter the data even more by just using the 99.5% or 99.0% of values

With this information we can state that the outliers only represent the 0.1% of values in the data, however, since the range is very large between the minimum and maximum values, we can say that the averages could be somewhat affected by this outliers. In this case we need to take that into acount by the time of sharing our insights

# 5.- Share

In [ ]:
yellow = '#e1f26b'
blue = '#4594a4'
green = '#9ccc84'
gray = '#d3e6ea'
lblue ='#9ccbd4'
white = '#FFFFFF'

### Outliers in ride length

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=3,figsize=(12,5),sharex=False)

plt.subplot(1,3,1)
ax1 = sns.boxplot(data=df,x='rideable_type',y='ride_length_min')
ax1.set_title('Ride length distribution per rideable type')

# Filtering out the docked_bike value
no_docked_bike = df.query('rideable_type != "docked_bike"')

plt.subplot(1,3,2)
ax2 = sns.boxplot(data = no_docked_bike, x='rideable_type', y='ride_length_min')
ax2.set_title('Ride length distribution without docked bike')

# Filtering our the 1% of values
percent_99 = np.percentile(df['ride_length_min'],99)
df_99 = df.loc[(df['ride_length_min'] <= percent_99)]

plt.subplot(1,3,3)
ax3 = sns.boxplot(data=df_99,x='rideable_type',y='ride_length_min')
ax3.set_title('Ride length distribution 99%')

plt.tight_layout()
plt.show()

We can see that for the type of docked bike, the ride length in minutes get way higher than those for the classic and electric bike, so the data is biased with many outliers.

For the classic bike and electric bike it seems there are not that many outliers, but after a closer look, we see in the second graph that there are many outliers as well, ranging up to 1600 minutes.

In the third graph we eliminate the 1% of values which exceed the normal distribution, so that we can see better the distribution and percentiles. Also confirms visually what we already know about the median of the ride length for the rideable type.

* 12 minutes for the classic bike
* 28 minutes for the docked bike
* 10 minutes for the electric bike

Also close to the average if we eliminate that 0.1% or up to 1% of very extreme values. These lengths are closer to what an actual user is expected to spend riding a bike every time they use the service.   

### Ride length per weekday

Let's see what is the difference between users in terms of ride length per day

In [ ]:
weekday_name = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

fig, ax= plt.subplots(1,1)
day_grp = df_99.groupby(['member_casual','day_of_week']).mean().reset_index()
sns.lineplot(data=day_grp, x='day_of_week',y='ride_length_min',hue='member_casual',palette=[green,blue],sizes=[20,20])
ax.set_xticks(range(len(weekday_name)))
ax.set_xticklabels(weekday_name)

plt.title('Ride length per weekday')
plt.grid(visible=True,alpha=0.3)

plt.show()

We can see that the casual users average ride length fall during weekdays and rise during the weekends.

The member users although also having an increase during the weekends, during weekdays(monday to friday) the tendency is flat around 12 minutes on average.

### Percentage of member and casual users 

What is the proportion of member and casual users from the total?

In [ ]:
users = df['member_casual']

plt.pie(users.value_counts(),labels=users.unique(),colors=[blue,green],autopct="%1.1f%%")

plt.title('Number of users per user type')
plt.tight_layout()
plt.show()

### Number of rides per user type

How many rides each type of user did? and for every type of rideable?

In [ ]:
ride_user_grp = df.groupby(['member_casual','rideable_type'])['ride_id'].count().reset_index(name='count')

ax = sns.barplot(data=ride_user_grp,x='rideable_type',y='count',hue='member_casual',palette=[green,blue],hue_order=['casual','member'])
                
plt.ylabel('Number of rides (millions)')

plt.legend()
plt.title('Number of rides per user type and rideable type')
plt.tight_layout()

plt.show()

We can see that the members ride more often than casual members in both classic and electric bikes, and we also see a lack of usage of the docked bikes by members

### Number of rides per month

What are the tendencies in the number of rides each type of user did each month?

In [ ]:
months = ['','January','Febraury','March','April','May','June','July','August','September','October','November','December']

fig, ax =plt.subplots(1,1,figsize=(9,6))

df['month'].value_counts().sort_index().plot(kind='area', label='all users',color=gray,alpha=1)
df.query('member_casual == "casual"')['month'].value_counts().sort_index().plot(kind='area', label='casual user',color=green,alpha=1)
df.query('member_casual == "member"')['month'].value_counts().sort_index().plot(kind='area', label='member user',color=blue,alpha=0.7)

ax.set_xticks(range(len(months)))
ax.set_xticklabels(months)

plt.xlim([1,12])
plt.xticks(rotation=20)
plt.legend()
plt.xlabel('Month')
plt.ylabel('Number of rides')
plt.tight_layout()
plt.show()

For all type of users, we see the rise in spring months, the spike during the summer months. And the fall entering autumn and the bottom at winter.

But we also see that the members tend to maintain the number of rides for the summer and autumn seasons. And on the contrary, the casual users take the lead in the mid-summer and then the number of rides fall faster than those for members.

### Number of rides per weekday

Are there any tendencies in the number of rides per each day of the week?

In [ ]:
import matplotlib.patches as mpatches

fig, ax =plt.subplots(1,1,figsize=(9,6))

ride_per_day = df['started_at'].dt.dayofweek.value_counts().sort_index(ascending=True).to_frame()
user_day = user_grp['day_of_week'].value_counts().rename_axis(['member_casual','day_of_week']).reset_index(name='count')

user_palette = {'casual':'tab:green', 'member':'tab:blue'}
plt.bar(ride_per_day.index,ride_per_day['started_at'],color=gray)
sns.barplot(data=user_day,x='day_of_week',y='count',hue='member_casual',palette=[green,blue],hue_order=['casual','member'])

ax.set_xticks(range(len(weekday_name)))
ax.set_xticklabels(weekday_name)

all_legend = mpatches.Patch(color=(gray), label='All')
casual_legend = mpatches.Patch(color=(green), label='casual')
member_legend = mpatches.Patch(color=(blue),label='member')

plt.legend(handles=[all_legend,casual_legend,member_legend])

plt.title('Number of rides per day')

We can observe some tendencies. In aggregate for all users and specially for casual users, the number of rides increase the weekends. But for members the tendency is flat and constant during the whole week, even drops slightly during weekends

### Maps

Finally I will show two maps to try answering the question: In what areas do casual and member users tend to ride?

In [ ]:
import plotly.express as px

df_sample = df.sample(n=5000)
df_20 = df.dropna().sample(n=100).sort_values(by=['member_casual','started_at']).reset_index(drop=True)

fig_1= px.scatter_mapbox(df_sample,lat='start_lat',lon='start_lng',color='member_casual',opacity=1,zoom=9,
                         color_discrete_sequence=[lblue,green],title='User clustering')

#fig = px.line_geo(df_20,lat='start_lat',lon='start_lng')

fig_1.update_layout(mapbox_style='carto-positron')

fig_1.show()

print()

fig2 = px.scatter_mapbox(df_20, lat="start_lat", lon="start_lng",
                        hover_name=df_20.index,opacity=1,zoom=9,title='User paths')
fig2.update_layout(mapbox_style="carto-positron")
#fig2.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

tsp = list(range(1,100))

fig2.add_traces(px.line_mapbox(df_20.loc[tsp], lat="start_lat", lon="start_lng",color='member_casual',color_discrete_sequence=[green,lblue]).data)
fig2.show()

You can see in the first map, that casual users tend to cluster in the city center, whereas the member users are more scattered in the outskirts of the city.

In the second map we can see the paths taken from users out of a random sample of the Data. This map shows a somewhat regular path the member users take, in and out the city center. By contrast, casual users take more irregular paths all around the city.

This information could imply that although both users share some paths and cluster in similar areas, casual users tend to ride more in leisure areas where there are parks and sometimes to explore all around the city, and member users follow more straightforward paths from the suburbs into the city center and vice-versa, meaning they might use it more for commuting to work

# 6.- Act

### Conclusion

With these findings we can give some insights and answer the question to the core business task: Indentify the differences between "Casual" and "Member" users.

1. Casual users ride on average longer than member users, but member users ride length doesn't fluctuate as much during the week.
2. Member users ride more often than casual riders, except on weekends and during the peak of the summer.
3. Member users follow predictable paths from and to the city center, whereas Casual users ride closer to leisure areas and follow less predictable paths.

Without more information about the users, like age, job, or gender, we can only infer by the data provided.

### Suggestion

I would suggest focusing on targeting casual users, showing the benefits of becoming a member user, such as making a good environmental impact by using bike sharing to commute to work instead of using other types of transportation, or how much money they can save by becoming members and still use the service for both leisure and other type of activities.

Thanks for reading my case study. I hope you found it useful, and any comments are welcome.